# IPv4 moodle quizz generator

The last sections of the notebook contains the code for generating various categories of questions/quizz in the Moodle format.

* IP address plan
* Subnet splitting
* Routing table (not implemented)
* Routing

Simply run all the cells.


GPLv2 Licence https://docs.moodle.org/19/fr/Licence

[XML Moodle output format support](https://docs.moodle.org/3x/fr/Question_cloze_à_réponses_intégrées)

# IP API

## Format utils

(bin, dec, dotted, CIDR)

In [2]:
from random import *
# import re

def dec_to_bin(d):
  #bin_value = bin(generate_ipv4_address())
  #
  #     if n==0: return ''
  #     else:
  #         return dec_to_bin(n/2) + str(n%2)
  return "{0:b}".format(d)

def bin_to_dec(b):
  return str(int(b, 2))
# import re

def split_dotted_dec_cidr (d):
  elements = d.split ('/')
  return elements[0], int(elements[1])

def get_mask_bin(mask_cidr):
  return '{}{}'.format(mask_cidr * '1', '0' * int(32-(mask_cidr)))

def mask_dec(mask_cidr):
  return pretty_dotted_dec(get_mask_bin(mask_cidr))

def dotted_dec_to_bin (d):
  octets = d.split ('.')
  print (octets)
  #bin_ip = list()
  bin_ip = ""
  for o in octets:
    bin_o = dec_to_bin(int(o))
    bin_ip += '0'* int(8-len(bin_o)) + bin_o
    #bin_ip.append('0'* int(8-len(bin_o)) + bin_o)
  print (bin_ip)
  return str(bin_ip) # '.'.join(bin_ip)



## Pretty

In [3]:
def pretty_dotted_bin(b, mask_cidr = ''):
  bin = b[0:4]+' '+ b[4:8]+ ' . '+ b[8:12]+' '+ b[12:16]+ ' . '+  b[16:20]+' '+ b[20:24]+ ' . '+ b[24:28]+' '+ b[28:32]
  if mask_cidr != '':
    offset = 0
    index = 0
    #print ('Debug: mask_cidr', mask_cidr)
    #print ('Debug: len(bin)', len(bin))

    while offset != mask_cidr:
      #print (offset, index, ">{}<".format(bin[index]))
      if bin[index] == '0' or bin[index] == '1':
        offset += 1
      index += 1
    bin = bin[:index] + '/' + bin[index:]
  return bin

def pretty_dotted_network_id_bin(b):
  index = 1
  pretty_b = list()
  while index < len(b)+1:
    pretty_b.append(b[index-1])
    if index%4 == 0 and len(b) != index:
      pretty_b.append(' ')
    if index%8 == 0 and len(b) != index:
      pretty_b.append('. ')
    index +=1
  return ''.join(pretty_b)

def pretty_dotted_dec(ip_bin, mask_cidr = ''):
  if mask_cidr != '':
    mask_cidr = '/'+ str(mask_cidr)
  return bin_to_dec(ip_bin[0:8])+'.'+bin_to_dec(ip_bin[8:16])+'.'+bin_to_dec(ip_bin[16:24])+'.'+bin_to_dec(ip_bin[24:32]) + mask_cidr

def pretty_hosts_length (mask):
  return "2^"+str(32-mask)+"-2"

## Internet

In [4]:
def get_address_class(b):
  address_class = "unk"
  if b.startswith("0"): return "A"
  if b.startswith("10"): return "B"
  if b.startswith("110"): return "C"
  return address_class

def network_id (ipv4_address_bin, mask):
  return ipv4_address_bin[0:mask]

def get_network_address_bin (ipv4_address_bin, mask):
  return network_id (ipv4_address_bin, mask)+ '0'* int(32-mask)

def get_broadcast_address_bin (ipv4_address_bin, mask):
  return network_id (ipv4_address_bin, mask)+ '1'* int(32-mask)

def is_network_address_bin (ipv4_address_bin, mask):
  return ipv4_address_bin == get_network_address_bin (ipv4_address_bin, mask)

def is_broadcast_address_bin (ipv4_address_bin, mask):
  return ipv4_address_bin == get_broadcast_address_bin (ipv4_address_bin, mask)

def is_host_address_bin (ipv4_address_bin, mask):
  return not (is_network_address_bin(ipv4_address_bin, mask) or is_broadcast_address_bin(ipv4_address_bin, mask))


def get_first_host_address_bin (ipv4_address_bin, mask):
  #print (network_id (ipv4_address_bin, mask))
  #print ('0' * int(32-(mask+1)))
  #print ('0' * int(32-(mask)))
  #print (1 * '1')
  return '{}{}{}'.format(network_id (ipv4_address_bin, mask), '0' * int(32-(mask+1)), 1 * '1')

def get_last_host_address_bin (ipv4_address_bin, mask):
  return '{}{}{}'.format(network_id (ipv4_address_bin, mask), '1' * int(32-(mask+1)), 1 * '0')

def hosts_length (mask):
  return (2**(32-mask))-2

def is_private_address (ipv4_address_bin, address_class):
  if address_class == 'A':
    # 10.0.0.0 - 10.255.255.255
    if ipv4_address_bin.startswith('00001010'): return True
  else:
    if address_class == 'B':
      # 172.16.0.0 - 172.31.255.255
      if ipv4_address_bin.startswith('10101100') and int("{0:b}".format(0), 2) <= int(ipv4_address_bin[8:16], 2) and int("{0:b}".format(31), 2) >= int(ipv4_address_bin[8:16], 2):
        return True
    else:
      if address_class == 'C':
      # 192.168.0.0 - 192.168.255.255
        if ipv4_address_bin.startswith('1100000010101000'):
          #and int("{0:b}".format("0"), 2) <= int(ipv4_address_bin[8:16], 2)
          #and int("{0:b}".format("31"), 2) >= int(ipv4_address_bin[8:16], 2):
          return True
  return False



def are_on_the_same_network(ad1, ad2, mask_cidr):
  return ad1[0:mask_cidr] == ad2[0:mask_cidr]


def binary_length (number):
  return len (dec_to_bin(number))

def host_id (ip_address_bin, mask, ip_addr_len = 32):
  return ip_address_bin[mask:ip_addr_len]

def get_host_id_bin(ip_address_bin, mask, ip_addr_len = 32):
  return host_id (ip_address_bin, mask, ip_addr_len = 32)

def get_network_id_bin (ipv4_address_bin, mask):
  return network_id (ipv4_address_bin, mask)

def get_last_subnet_address_bin (addr_initial, mask_initial, subnet_mask, number_of_subnets):
        last_required_subnet_id = dec_to_bin(number_of_subnets-1)
        #print ('last_required_subnet_id', last_required_subnet_id)

        network_id_initial =  network_id (addr_initial, mask_initial)

        last_subnet_network_addr = network_id_initial + last_required_subnet_id + '0' * (32 - subnet_mask)
        #print ('last_network_addr', last_subnet_network_addr, 'len(last_network_addr)', len(last_subnet_network_addr))
        return last_subnet_network_addr


## Generate

In [5]:
def generate_ipv4_address_bin():
    d = randint(1,2**32) # importance d'avoir 32 bits donc pas une bonne solution à moins de bufferiser
    b = dec_to_bin(d)
    #print ("debug:"+ b)
    b = '0'* int(32-len(b)) + b
    #print ("debug:"+ b)
    return b


def generate_mask_cidr(min = 4, max = 28):
    # on s'interdit 32 (classe D), 31 (peu de sens), 0 (adresse d'initialisation)
    #n = randint(1,30)
    #24  256
    #23  512
    #22  1024
    #21  2048
    #20  4096
    n = randint(min, max)
    return n

## Use case: test are on the same network

def generate_close_host(ipv4_address_bin, mask_cidr):
  id = network_id (ipv4_address_bin, mask_cidr)
  id = id[0:len(id)-1] # -1 on tire aléatoire un nombre si premier bit tiré est identique à celui supprimé alors appartiendra et sinon non...
  nb_bits_machine_plus_1 = 32- len(id)
  d = randint(1,2**(nb_bits_machine_plus_1)) # importance d'avoir 32 bits donc pas une bonne solution à moins de bufferiser
  b = dec_to_bin(d)
  #print ("debug:"+ b)
  b = id + b + '0'* int(32- (len(id) + len(b)) -1) + '1'
  return b

## Subnet

def generate_number_of_subnets (min = 5, max = 4096):
  # on souhaite que le nombre de bits min requis soit >= 3 et <= 12
  return randint(min, max)

def generate_number_of_hosts (min = 5, max = 4096):
  # on souhaite que le nombre de bits min requis soit >= 3 et <= 12
  return randint(min, max)



## Test

Method to print all the information in order to debug

In [6]:

def test():

  ipv4_address_bin = generate_ipv4_address_bin()
  mask_cidr = generate_mask_cidr()

  # d 172.17.2.17 ; B /16 privé ; machine
  # i 10.255.255.255 ; A /8 ; privée (10...) ; adresse de diffusion
  # e 172.32.9.2 ; B /16 publique ; machine

  #ipv4_address_dotted_dec = "172.17.2.17"
  #mask_cidr = 16
  # SET
  #ipv4_address_dotted_dec, mask_cidr = split_dotted_dec_cidr("192.168.123.202/23")
  #ipv4_address_dotted_dec, mask_cidr = split_dotted_dec_cidr("128.201.168.143/27")
  #ipv4_address_bin = dotted_dec_to_bin(ipv4_address_dotted_dec)

  ipv4_address_dec_pte = pretty_dotted_dec(ipv4_address_bin)
  address_class = get_address_class(ipv4_address_bin)

  network_address_bin = get_network_address_bin(ipv4_address_bin, mask_cidr)
  broadcast_address_bin = get_broadcast_address_bin(ipv4_address_bin, mask_cidr)
  first_host_address_bin = get_first_host_address_bin(ipv4_address_bin, mask_cidr)
  last_host_address_bin = get_last_host_address_bin(ipv4_address_bin, mask_cidr)

  close_host_bin = generate_close_host(ipv4_address_bin, mask_cidr)
  close_host_dec_pte = pretty_dotted_dec(close_host_bin)


  #print(ipv4_address_bin)
  print("@ip binaire pointée\t\t"+pretty_dotted_bin(ipv4_address_bin))
  #print (ipv4_address_bin[0:8])
  print("@ip décimal pointée\t\t"+ipv4_address_dec_pte)

  print("address_class\t"+address_class)

  print ("is private ", is_private_address(ipv4_address_bin, address_class))
  print ("is network_address", is_network_address_bin(ipv4_address_bin, mask_cidr))
  print ("is broadcast_address", is_broadcast_address_bin(ipv4_address_bin, mask_cidr))
  print ("is host_address", is_host_address_bin(ipv4_address_bin, mask_cidr))

  print("mask_cidr\t"+str(mask_cidr))
  print ("mask_dec\t"+mask_dec(mask_cidr))


  print("@réseau binaire pointée\t\t"+pretty_dotted_bin(network_address_bin))
  print("@réseau décimal pointée\t\t"+pretty_dotted_dec(network_address_bin))
  print("@réseau décimal pointée/mask_cidr\t\t"+pretty_dotted_dec(network_address_bin, mask_cidr))


  print("@diffusion binaire pointée\t"+pretty_dotted_bin(broadcast_address_bin))
  print("@diffusion décimal pointée\t"+pretty_dotted_dec(broadcast_address_bin))

  print("#hosts\t"+str(hosts_length(mask_cidr)))
  print("#hosts\t"+str(pretty_hosts_length(mask_cidr)))

  print ("plage d'adresse")


  print("@de binaire pointée\t\t"+pretty_dotted_bin(first_host_address_bin))
  print("@de décimal pointée\t\t"+pretty_dotted_dec(first_host_address_bin))
  print("@à binaire pointée\t\t"+pretty_dotted_bin(last_host_address_bin))
  print("@à décimal pointée\t\t"+pretty_dotted_dec(last_host_address_bin))

  print("@close host binaire pointée\t"+pretty_dotted_bin(close_host_bin))
  print("@close host décimal pointée\t"+pretty_dotted_dec(close_host_bin))

  print('are_on_the_same_network (',pretty_dotted_dec(ipv4_address_bin),',',pretty_dotted_dec(close_host_bin),')=', are_on_the_same_network(ipv4_address_bin, close_host_bin, mask_cidr))

  print()
  number = 8
  print ('number=',number)
  print ('dec_to_bin(number)=',dec_to_bin(number))
  print ('binary_length (number)=len (dec_to_bin(number))=',binary_length (number))

  print()

test()


@ip binaire pointée		1111 0101 . 0110 1001 . 1011 1010 . 1110 1011
@ip décimal pointée		245.105.186.235
address_class	unk
is private  False
is network_address False
is broadcast_address False
is host_address True
mask_cidr	20
mask_dec	255.255.240.0
@réseau binaire pointée		1111 0101 . 0110 1001 . 1011 0000 . 0000 0000
@réseau décimal pointée		245.105.176.0
@réseau décimal pointée/mask_cidr		245.105.176.0/20
@diffusion binaire pointée	1111 0101 . 0110 1001 . 1011 1111 . 1111 1111
@diffusion décimal pointée	245.105.191.255
#hosts	4094
#hosts	2^12-2
plage d'adresse
@de binaire pointée		1111 0101 . 0110 1001 . 1011 0000 . 0000 0001
@de décimal pointée		245.105.176.1
@à binaire pointée		1111 0101 . 0110 1001 . 1011 1111 . 1111 1110
@à décimal pointée		245.105.191.254
@close host binaire pointée	1111 0101 . 0110 1001 . 1011 1110 . 1001 0110
@close host décimal pointée	245.105.190.150
are_on_the_same_network ( 245.105.186.235 , 245.105.190.150 )= True

number= 8
dec_to_bin(number)= 1000
binar

In [7]:
def debug(ipv4_address_bin, mask_cidr, ipv4_address_dec_pte, address_class, network_address_bin, broadcast_address_bin, first_host_address_bin, last_host_address_bin, close_host_bin, close_host_dec_pte):

  #print(ipv4_address_bin)
  print("@ip binaire pointée\t\t"+pretty_dotted_bin(ipv4_address_bin))
  #print (ipv4_address_bin[0:8])
  print("@ip décimal pointée\t\t"+ipv4_address_dec_pte)

  print("address_class\t"+address_class)

  print ("is private ", is_private_address(ipv4_address_bin, address_class))
  print ("is network_address", is_network_address_bin(ipv4_address_bin, mask_cidr))
  print ("is broadcast_address", is_broadcast_address_bin(ipv4_address_bin, mask_cidr))
  print ("is host_address", is_host_address_bin(ipv4_address_bin, mask_cidr))

  print("mask_cidr\t"+str(mask_cidr))
  print ("mask_dec\t"+mask_dec(mask_cidr))


  print("@réseau binaire pointée\t\t"+pretty_dotted_bin(network_address_bin))
  print("@réseau décimal pointée\t\t"+pretty_dotted_dec(network_address_bin))


  print("@diffusion binaire pointée\t"+pretty_dotted_bin(broadcast_address_bin))
  print("@diffusion décimal pointée\t"+pretty_dotted_dec(broadcast_address_bin))

  print("#hosts\t"+str(hosts_length(mask_cidr)))
  print("#hosts\t"+str(pretty_hosts_length(mask_cidr)))

  print ("plage d'adresse")


  print("@de binaire pointée\t\t"+pretty_dotted_bin(first_host_address_bin))
  print("@de décimal pointée\t\t"+pretty_dotted_dec(first_host_address_bin))
  print("@à binaire pointée\t\t"+pretty_dotted_bin(last_host_address_bin))
  print("@à décimal pointée\t\t"+pretty_dotted_dec(last_host_address_bin))

  print("@close host binaire pointée\t"+pretty_dotted_bin(close_host_bin))
  print("@close host décimal pointée\t"+pretty_dotted_dec(close_host_bin))

  print()


# Moodle quizz format API

* See https://docs.moodle.org/403/en/Embedded_Answers_(Cloze)_question_type
* https://docs.moodle.org/403/en/Import_questions

### format a quizz taking into input questions as a string

In [8]:
def format_moodle_quizz (questions_string):
  # in xml format

  quizz = list()
  quizz.append('<?xml version="1.0" encoding="UTF-8"?>')
  quizz.append('<quiz>')
    #('<!-- question: 0  -->\
    #<question type="category">\
      #<category>\
        #<text>top</text></category>\
    #</question>\
    #<!-- question: 0  -->\
    #<question type="category">\
      #<category>\
        #<text>top/Défaut pour M2102</text>\
      #</category>\
    #</question>')
  quizz.append(questions_string)
  quizz.append('</quiz>')
  return ''.join(quizz)

### format a cloze question

In [9]:
def format_cloze_question (name, questiontext, generalfeedback, hint):
  # string for each input,
  # output: in xml format

  cloze_question = list()
  cloze_question.append('<question type="cloze">')

  cloze_question.append('<name><text>')
  cloze_question.append(name)
  cloze_question.append('</text></name>')

  cloze_question.append('<questiontext format="html"><text><![CDATA[<p>')
  cloze_question.append(questiontext)
  cloze_question.append('<br><br></p>]]></text></questiontext>')

  if len(generalfeedback) != 0:
    cloze_question.append ('<generalfeedback format="html"><text><![CDATA[<p>')
    cloze_question.append(generalfeedback)
    cloze_question.append('</p>]]></text></generalfeedback>')
    cloze_question.append ('<penalty>0.3333333</penalty><hidden>0</hidden>') # semble être requis pour gérer les indices

  if len(hint) != 0:
    cloze_question.append ('<hint format="html"><text><![CDATA[<p>')
    cloze_question.append(hint)
    cloze_question.append ('</p>]]></text>')
    cloze_question.append ('</hint>')

    # indice 2 : indique où il y a une erreur
    cloze_question.append ('<hint format="html">')
    cloze_question.append ('<text><![CDATA[<p><br></p>]]></text>')
    cloze_question.append ('<shownumcorrect/>')
    cloze_question.append ('<clearwrong/>')
    cloze_question.append ('</hint>')

  cloze_question.append('</question>')
  return ''.join(cloze_question)

### format a shortanswer question

In [10]:
def format_shortanswer_question(text_before,answer,text_after):
  shortanswer_question = list()
  shortanswer_question.append(text_before)
  shortanswer_question.append('{1:SHORTANSWER:%100%')
  shortanswer_question.append(answer)
  shortanswer_question.append('}')
  shortanswer_question.append(text_after)
  shortanswer_question.append ("<br>")
  return ''.join(shortanswer_question)

### format a multichoice question

In [11]:
def format_multichoice_question(text_before,alternative_answers,true_one,text_after):
  # is a list of candidate answers, and true_one indicate the rank of the true one
  multichoice_question = list()
  multichoice_question.append(text_before)
  multichoice_question.append('{1:MC:')
  alternative_answers[true_one] = '%100%'+alternative_answers[true_one]
  multichoice_question.append('~'.join(alternative_answers))
  multichoice_question.append('}')
  multichoice_question.append(text_after)
  multichoice_question.append ("<br>")
  return ''.join(multichoice_question)

### format a multichoice question horizontal

In [12]:
def format_multichoice_horizontal_question(text_before,alternative_answers,true_one,text_after):
  # is a list of candidate answers, and true_one indicate the rank of the true one
  multichoice_question = list()
  multichoice_question.append(text_before)
  multichoice_question.append('{1:MCH:')
  alternative_answers[true_one] = '%100%'+alternative_answers[true_one]
  multichoice_question.append('~'.join(alternative_answers))
  multichoice_question.append('}')
  multichoice_question.append(text_after)
  multichoice_question.append ("<br>")
  return ''.join(multichoice_question)

# Quizz generation

## IP address plan

method to generate address plans

In [13]:
def generate_plan ():
  ipv4_address_bin = generate_ipv4_address_bin()
  mask_cidr = generate_mask_cidr()

  # d 172.17.2.17 ; B /16 privé ; machine
  # i 10.255.255.255 ; A /8 ; privée (10...) ; adresse de diffusion
  # e 172.32.9.2 ; B /16 publique ; machine

  #ipv4_address_dotted_dec = "172.17.2.17"
  #mask_cidr = 16
  # SET
  #ipv4_address_dotted_dec, mask_cidr = split_dotted_dec_cidr("192.168.123.202/23")
  #ipv4_address_dotted_dec, mask_cidr = split_dotted_dec_cidr("128.201.168.143/27")
  #ipv4_address_bin = dotted_dec_to_bin(ipv4_address_dotted_dec)

  ipv4_address_dec_pte = pretty_dotted_dec(ipv4_address_bin)
  address_class = get_address_class(ipv4_address_bin)

  network_address_bin = get_network_address_bin(ipv4_address_bin, mask_cidr)
  broadcast_address_bin = get_broadcast_address_bin(ipv4_address_bin, mask_cidr)
  first_host_address_bin = get_first_host_address_bin(ipv4_address_bin, mask_cidr)
  last_host_address_bin = get_last_host_address_bin(ipv4_address_bin, mask_cidr)

  close_host_bin = generate_close_host(ipv4_address_bin, mask_cidr)
  close_host_dec_pte = pretty_dotted_dec(close_host_bin)

  #debug(ipv4_address_bin, mask_cidr, ipv4_address_dec_pte, address_class, network_address_bin, broadcast_address_bin, first_host_address_bin, last_host_address_bin, close_host_bin, close_host_dec_pte)

  question = list()
  if address_class in ['A', 'B', 'C']:
    if wi_html:question.append ('<question type="cloze"><name><text>cloze plan '+ipv4_address_dec_pte+"/"+str(mask_cidr)+'</text></name><questiontext format="html"><text><![CDATA[<p>')
    else: question.append ("\n")

    question.append ("Soit l'adresse IP et le masque suivants : "+ipv4_address_dec_pte+"/"+str(mask_cidr)+".")
    if wi_html:  question.append ("<br>")
    else: question.append ("\n")

    question.append ("En décimal pointée (W.X.Y.Z) le masque CIDR aurait pour valeur {1:SHORTANSWER:%100%"+mask_dec(mask_cidr)+"}.")
    if wi_html:  question.append ("<br>")
    else: question.append ("\n")

    if address_class == 'A':
      #question.append ("Dans le système de classe, cette adresse serait de classe (A, B ou C) : {1:SHORTANSWER:%100%"+address_class+"} ")
      question.append ("Si le système de classes était toujours d'actualités, cette adresse serait de classe {1:MC:%100%A~B~C} (A, B ou C).")
    else:
      if address_class == 'B':
        question.append ("Si le système de classes était toujours d'actualités, cette adresse serait de classe {1:MC:A~%100%B~C} (A, B ou C).")
      else:
        if address_class == 'C':
          question.append ("Si le système de classes était toujours d'actualités, cette adresse serait de classe {1:MC:A~B~%100%C} (A, B ou C).")
    if wi_html:  question.append ("<br>")
    else: question.append ("\n")

    if is_private_address(ipv4_address_bin, address_class):
      question.append ("Cette adresse est {1:MC:%100%privée~publique} (privée ou publique).")
    else:
      question.append ("Cette adresse est {1:MC:privée~%100%publique} (privée ou publique).")
    if wi_html:  question.append ("<br>")
    else: question.append ("\n")

    if is_network_address_bin(ipv4_address_bin, mask_cidr):
      question.append ("Cette adresse est une adresse de {1:MC:%100%réseau~diffusion~machine} (réseau, diffusion, machine).")
    else:
      if is_broadcast_address_bin(ipv4_address_bin, mask_cidr):
        question.append ("Cette adresse est une adresse de {1:MC:réseau~%100%diffusion~machine} (réseau, diffusion, machine).")
      else:
        if is_host_address_bin(ipv4_address_bin, mask_cidr):
          question.append ("Cette adresse est une adresse de {1:MC:réseau~diffusion~%100%machine} (réseau, diffusion, machine).")
    if wi_html:  question.append ("<br>")
    else: question.append ("\n")

    question.append ("L'adresse réseau est {1:SHORTANSWER:%100%"+pretty_dotted_dec(network_address_bin)+"} (en décimal pointée W.X.Y.Z).")
    if wi_html:  question.append ("<br>")
    else: question.append ("\n")

    question.append ("L'adresse de diffusion est {1:SHORTANSWER:%100%"+pretty_dotted_dec(broadcast_address_bin)+"} (en décimal pointée W.X.Y.Z).")
    if wi_html:  question.append ("<br>")
    else: question.append ("\n")

    question.append ("Sur ce réseau, on peut adresser {1:SHORTANSWER:%100%"+str(hosts_length(mask_cidr))+"} machines.")
    if wi_html:  question.append ("<br>")
    else: question.append ("\n")

    question.append ("L'adresse IP de la première machine adressable est {1:SHORTANSWER:%100%"+pretty_dotted_dec(first_host_address_bin)+"} (en décimal pointée W.X.Y.Z).")
    if wi_html:  question.append ("<br>")
    else: question.append ("\n")

    question.append ("L'adresse IP de la dernière machine adressable est {1:SHORTANSWER:%100%"+pretty_dotted_dec(last_host_address_bin)+"} (en décimal pointée W.X.Y.Z).")
    if wi_html:  question.append ("<br>")
    else: question.append ("\n")

    if are_on_the_same_network(ipv4_address_bin, close_host_bin, mask_cidr):
      question.append ("Est ce que l'adresse IP "+close_host_dec_pte+" se trouve sur le même réseau ? {1:MC:%100%oui~non} (oui ou non).")
    else:
      question.append ("Est ce que l'adresse IP "+close_host_dec_pte+" se trouve sur le même réseau ? {1:MC:oui~%100%non} (oui ou non).")
    if wi_html:  question.append ('<br><br></p>]]></text></questiontext><generalfeedback format="html"><text><![CDATA[<p>')

    if wi_hints:
      #print(ipv4_address_bin)
      question.append("@IP en binaire pointée\t\t:"+pretty_dotted_bin(ipv4_address_bin))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")
      #print (ipv4_address_bin[0:8])
      question.append("@IP en décimal pointée\t\t:"+ipv4_address_dec_pte)
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append("Classe d'adresse : \t"+address_class)
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append ("Est privée : "+ str(is_private_address(ipv4_address_bin, address_class)))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")
      question.append ("Est une adresse réseau : "+ str(is_network_address_bin(ipv4_address_bin, mask_cidr)))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")
      question.append ("Est une adresse de diffusion : "+ str(is_broadcast_address_bin(ipv4_address_bin, mask_cidr)))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")
      question.append ("Est une adresse machine : "+ str(is_host_address_bin(ipv4_address_bin, mask_cidr)))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append("Le masque en notation CIDR est \t:"+str(mask_cidr))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")
      question.append("Le masque en binaire pointée est \t:"+pretty_dotted_bin(get_mask_bin(mask_cidr)))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")
      question.append ("Le masque en décimal pointée est \t:"+mask_dec(mask_cidr))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")




      question.append("@réseau en binaire pointée\t\t:"+pretty_dotted_bin(network_address_bin))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")
      question.append("@réseau en décimal pointée\t\t:"+pretty_dotted_dec(network_address_bin))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")


      question.append("@diffusion en binaire pointée\t:"+pretty_dotted_bin(broadcast_address_bin))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")
      question.append("@diffusion en décimal pointée\t:"+pretty_dotted_dec(broadcast_address_bin))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append("#hosts\t"+str(pretty_hosts_length(mask_cidr))+' = '+str(hosts_length(mask_cidr)))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append ("Plage d'adresse :")
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append("@de en binaire pointée\t\t:"+pretty_dotted_bin(first_host_address_bin))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")
      question.append("@de en décimal pointée\t\t:"+pretty_dotted_dec(first_host_address_bin))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")
      question.append("@à en binaire pointée\t\t:"+pretty_dotted_bin(last_host_address_bin))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")
      question.append("@à en décimal pointée\t\t:"+pretty_dotted_dec(last_host_address_bin))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append("@IP autre machine en binaire pointée\t:"+pretty_dotted_bin(close_host_bin))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")
      question.append("@IP autre machine en décimal pointée\t:"+pretty_dotted_dec(close_host_bin))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append ("Sont sur le même réseau : " +str(are_on_the_same_network(ipv4_address_bin, close_host_bin, mask_cidr)))
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

    if wi_html:  question.append ('</p>]]></text></generalfeedback><penalty>0.3333333</penalty><hidden>0</hidden>')
    else: question.append ("\n")




    if wi_hints:
      # indice 1 : donne les versions binaires
      if wi_html:  question.append ('<hint format="html"><text><![CDATA[<p>')
      else: question.append ("\n")

      question.append ("Le principe général est de tout convertir en binaire, de faire les calculs en binaires puis de rendre les résultats en décimal.")
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append ("L'adresse IP en décimal pointée "+ipv4_address_dec_pte+" se convertit en binaire en "+pretty_dotted_bin(ipv4_address_bin)+".")
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append ("Pour connaître la classe d'adresses (système qui n'existe plus aujourd'hui et qui a été remplacé par la notation CIDR), il faut regarder la valeur binaire du 1er octet. Si le préfixe binaire (les bits de poids forts) est 0 alors c'est de classe A. Si 10 alors de classe B. Si 110 alors de classe C.")
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append ("Dans chaque classe, au moins une adresse de réseau (et toutes les adresses des machines sur ce réseau) a été réservée pour un usage privé (cad n'existe pas sur Internet). En classe A, 10.0.0.0 - 10.255.255.255. En classe B, 172.16.0.0 - 172.31.255.255. En classe C, 192.168.0.0 - 192.168.255.255")
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append ("Une adresse IP compte dans sa partie bit de poids fort (à gauche) l'identifiant du réseau et dans sa partie bit de poids faible (à droite) l'identifiant de la machine sur ce réseau. Le masque permet de déterminer la partie de l'adresse IP qui est réservée pour identifier le réseau et celle qui sert pour identifier la machine sur ce réseau. Par définition, un masque a les bits de la partie réseau à 1 et les bits de la partie machine à 0. Il peut être exprimé en notation CIDR (avec un slash suivi d'une valeur décimal qui indique le nombre de bits de poids fort utilisés pour coder l'identifiant réseau), on peut aussi transcrire la valeur du masque en décimal pointée (W.X.Y.Z).")
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append ("Pour déterminer l'adresse réseau (qui désigne le réseau), on fait un 'et logique' entre le masque et l'adresse IP en focus. Ou bien on met à zéro tous les bits de la partie machine de l'IP en focus.")
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append ("Pour déterminer l'adresse de diffusion (qui permet d'adresser toutes les machines de ce réseau), on met à un tous les bits de la partie machine de l'adresse IP en focus.")
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append ("Pour déterminer si on est en présence d'une adresse IP d'une machine, il s'agit de vérifier que l'adresse n'est ni une adresse de réseau ni une adresse de diffusion.")
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append ("Le nombre de machines adressables sur un réseau dépend du nombre de bits alloués à la partie machine sur l'adresse IP (soit '32-masque'). Cela se calcule à l'aide de la formule (2^(32-masque))-2. Le '-2' correspond aux adresses de réseau et de diffusion qui ne peuvent être utilisés pour adresser une machine.")
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append ("La plage d'adresses machines adressables s'étend de 'l'adresse réseau plus 1' en binaire à 'l'adresse de diffusion moins 1' en binaire.")
      if wi_html:  question.append ("<br>")
      else: question.append ("\n")

      question.append ("Pour déterminer si une machine est sur un réseau ou si deux machines sont sur le même réseau, alors on observe la partie réseau de ces IP. Si elle est identique, ces machines sont censées être sur le même réseau.")
      if wi_html: question.append ("<br>")
      else: question.append ("\n")

      if wi_html:  question.append ('</p>]]></text>')
      if wi_html:  question.append ('</hint>')

      # indice 2 : indique où il y a une erreur
      if wi_html:  question.append ('<hint format="html"><text><![CDATA[<p>')
      if wi_html:  question.append ('<br></p>]]></text>')
      if wi_html:  question.append ('<shownumcorrect/>')
      if wi_html:  question.append ('<clearwrong/>')
      if wi_html:  question.append ('</hint>')

    if wi_html:  question.append ('</question>')
    else: question.append ("\n")
    return question
  #print ("Dans le système de classe, cette adresse serait de classe: {1:MCS:=A#Bonne réponse~Arizona#Mauvaise réponse}")
  #    Appariez les villes et les états :
  #    San Francisco: {1:MCS:=Californie#Bonne réponse~Arizona#Mauvaise réponse}
  #    Tucson: {1:MCS:Californie#Mauvaise réponse~%100%Arizona#Bonne réponse}
  #    Los Angeles: {1:MCS:=Californie#Bonne réponse~Arizona#Mauvaise réponse}
  #    Phoenix: {1:MCS:%0%Californie#Mauvaise réponse~=Arizona#Bonne réponse}
  # La capitale de la France est {1:SHORTANSWER:%100%Paris#Bravo!~%50%Marseille#Non, c'est la deuxième plus grande ville de France (après Paris).~*#Mauvaise réponse. La capitale de la France est Paris, bien sûr.}.
  # 23+ 0.8 = {2:NUMERICAL:=23.8:0.1#Rétroaction pour la réponse correcte~%50%23.8:2#Rétroaction pour la réponse valant la moitié des points}.</pre>
  # Ces trois questions seraient intégrées dans UNE SEULE question cloze. (Sans sauts de ligne entre les { } !)



generate a quizz with hints (e.g. wi_hints = True), and the number of questions to generate (e.g. max_questions = 100)

In [14]:
count = 0
# SET
max_questions = 100
wi_feedback = True # not implemented
wi_hints = False
wi_html = True # True for xml moodle format export

questions = list()

while count < max_questions:
  question = generate_plan()
  if (question != None):
    #print ("no question")
    questions.append(''.join(question))
    count +=1

quizz_path = 'IP_address_plan_quizz.xml'
with open(quizz_path, "w") as text_file:
    if wi_html:
      questions.insert(0, '<?xml version="1.0" encoding="UTF-8"?><quiz><!-- question: 0  --><question type="category"><category><text>top</text></category></question><!-- question: 0  --><question type="category"><category><text>top/Défaut pour M2102</text></category></question>')
    if wi_html:questions.append ('</quiz>')
    text_file.write(''.join(questions))





## Subnet


* génère moi une IP et un masque

sur les 32-m restant

génère moi un nombre de réseaux entre ...

- combien de bits disponibles avec le masque initial donné ici  
- combien de bits pour coder nb réseaux
- combien de bits pour coder nb de machines
- est-ce possible de satisfaire les deux contrainte avec le masque initial donné ici
- quel est le nouveau masque si on privilégie le nombre minimal de bits nécessaires pour coder les ids réseaux ?
- combien d'hôtes sont disponibles pour le 1er sous-réseau
- est-ce que les autres sous-réseaux ont le même nombre d'hôtes adressables O/n
- donner en décimal pointée l'adresse réseau du 1er sous réseau
- donner en décimal pointée l'adresse diffision du 1er sous réseau
- donner en décimal pointée l'IP de la première machine adressable sur ce 1er  sous réseau
- donner en décimal pointée l'IP de la dernière machine adressable sur ce 1er sous réseau
- donner en décimal pointée l'IP de la première machine adressable sur ce dernier sous réseau demandé
- donner en décimal pointée l'IP de la dernière machine adressable sur ce dernier sous réseau demandé

* on décide qu'on ne travaillera qu'entre le 8e et le 32 bits
- tirage aléatoire de deux nombres compris entre 5 et 4096 qui correspondent respectivement à des valeurs binaires codés entre 3 et 12 bits.
* la somme des deux faits aux max 24 bits

In [15]:
def generate_subnet_splitting():

  mask_initial = generate_mask_cidr()
  #print ('mask_initial', mask_initial)
  #mask_initial = 18 # DEBUG

  # 80.123.192.0/18 .<BR>
  # On vous demande de découper ce réseau en 8 sous-réseaux d'au moins 140 hôtes chacun

  addr_initial = generate_ipv4_address_bin()
  #addr_initial = dotted_dec_to_bin('80.123.192.0') # DEBUG
  #print ('addr_initial', addr_initial, 'len(addr_initial)', len(addr_initial))

  network_address_bin = get_network_address_bin(addr_initial, mask_initial)

  address_class = get_address_class(addr_initial)

  question = list()
  generalfeedback = list()
  hint = list()
  name = ''
  if address_class in ['A', 'B', 'C']:

    available_bits = len(host_id(addr_initial, mask_initial))
    #print ('available_bits', available_bits)

    min = 3
    max = 3
    if available_bits > min+max:
      #print ('Warning: this run may be considered!')


      border = randint(min,available_bits-max)
      #print ('border',border)

      max_number_of_subnets = 2**border
      #print ('max_number_of_subnets',max_number_of_subnets)
      number_of_subnets = randint(5,max_number_of_subnets)
      #number_of_subnets = 8 # DEBUG
      binary_len_of_subnets = binary_length(number_of_subnets-1)  # on a besoin d'un nombre de bits pouvant coder le  number_of_subnets -1 valeurs (car on utilise 0 en binaire comme code)
      #print ('number_of_subnets', number_of_subnets, 'binary_len_of_subnets', binary_len_of_subnets)

      subnet_mask = mask_initial + binary_len_of_subnets

      max_number_of_hosts = 2**(available_bits-border) -2

      #print ('max_number_of_hosts',max_number_of_hosts)
      number_of_hosts = randint(min, max_number_of_hosts)
      #number_of_hosts = 140 # DEBUG
      binary_len_of_hosts = binary_length(number_of_hosts-1) # on a besoin d'un nombre de bits pouvant coder le  number_of_hosts -1 valeurs (car on utilise 0 en binaire comme code)
      #print ('number_of_hosts', number_of_hosts, 'binary_len_of_hosts', binary_len_of_hosts)
      actual_number_of_hosts = 2**(32-subnet_mask) -2
      #print ('Debug: number_of_hosts=',number_of_hosts)
      #print ('Debug: binary_len_of_hosts=',binary_len_of_hosts)
      #print ('Debug: subnet_mask=',subnet_mask)
      #print ('Debug: actual_number_of_hosts=',actual_number_of_hosts)



      # afin de rester dans le domaine du calculable...
      if binary_len_of_subnets <=13 and binary_len_of_hosts <= 13:

        question.append(''.join(["Soit le réseau IP suivant : ",pretty_dotted_dec(network_address_bin,mask_initial),".<BR>"]))

        question.append(format_shortanswer_question(
          "Combien de bits disposez-vous dans l'identifiant de l'hôte (aussi appelé identifiant machine) ? ",
          str(available_bits),
          "<BR>"))
        generalfeedback.append(''.join(['Le réseau ',pretty_dotted_dec(network_address_bin,mask_initial),' compte ', str(available_bits), " bits disponibles dans la partie identifiant de l'hôte.<BR>"]))
        hint.append(''.join([
          "Pour calculer le nombre de bits disponibles dans la partie identifiant de l'hôte, il suffit de soustraire le masque CIDR à 32, soit 32-",
          str(mask_initial),
          " (avec 32, la taille d'une adresse IPv4), ce qui donne ici ", str(available_bits), ' bits disponibles.<BR>']))


        question.append('<BR>')
        question.append(''.join(['On vous demande de découper ce réseau en ', str(number_of_subnets),
                      " sous-réseaux d'au moins ", str(number_of_hosts), ' hôtes chacun.<BR>']))

        question.append(format_multichoice_horizontal_question('Est-ce possible ? ', ['oui', 'non'], 0, '<BR>'))

        question.append(format_shortanswer_question(
          "Combien de bits sont nécessaires pour coder le nombre de sous-réseaux demandés ? ",
          str(binary_len_of_subnets),
          "<BR>"))

        question.append(format_shortanswer_question(
          "Combien de bits sont nécessaires pour coder le nombre d'hôtes demandés ? ",
          str(binary_len_of_hosts),
          "<BR>"))

        generalfeedback.append(''.join(['Il faut ', str(binary_len_of_subnets), ' bits pour coder ', str(number_of_subnets),' sous-réseaux.<BR>']))
        generalfeedback.append(''.join(['Il faut ', str(binary_len_of_hosts), ' bits pour coder ', str(number_of_hosts),' hôtes.<BR>']))
        generalfeedback.append(''.join([str(available_bits), ' bits sont donc suffisants pour couvrir ces besoins.<BR>']))
        hint.append(''.join(["Pour trouver le nombre de bits suffisants pour coder un nombre de sous-réseaux ou un nombre d'hôtes, il suffit de trouver la première puissance de 2 a avoir une valeur supérieure ou égale aux nombres demandées.<BR>"]))


        question.append(format_shortanswer_question(
          "Quelle valeur aura le nouveau masque ? ",
          str(subnet_mask),
          "(en notation CIDR sans le '/') <BR>"))
        generalfeedback.append(''.join(['Le masque de sous-réseau sera ', str(subnet_mask), ' (en notation CIDR).<BR>']))

        hint.append(''.join(["Le nouveau masque s'obtient en sommant la valeur CIDR du masque initial (ici ",str(mask_initial),
                           ') avec le nombre de bits requis pour coder le nombre de sous-réseaux demandés (ici ', str(binary_len_of_subnets),
                           '), ce qui donne ',str(subnet_mask),'.<BR>']))

        question.append(format_shortanswer_question(
          "Dans l'absolu, combien d'hôtes distincts peut-on adresser dans chaque sous-réseau ? ",
          str(actual_number_of_hosts),
          "<BR>"))
        generalfeedback.append(''.join(['On peut coder ', str(actual_number_of_hosts), ' hôtes distincts quand on applique ce masque.<BR>']))
        hint.append (''.join(["Le nombre de machines adressables sur un réseau dépend du nombre de bits alloués à la partie machine sur l'adresse IP (soit '32-masque').",
                     " Cela se calcule à l'aide de la formule (2^(32-masque))-2.",
                     " Le '-2' correspond aux adresses de réseau et de diffusion qui ne peuvent être utilisés pour adresser une machine.<BR>"]))
        hint.append(''.join(["Le nouveau masque que l'on retranche à 32 permet d'obtenir le nombre de bits disponibles dans l'identifiant de l'hôte, soit 32-",
                           str(subnet_mask),
                           " qui donne ", str(32-subnet_mask),
                           " bits disponibles. Pour calculer le nombre d'hôtes distincts que l'on peut adresser sur chaque sous-réseau il faut donc faire 2^",
                           str(32-subnet_mask),
                           "-2 qui donne ", str(actual_number_of_hosts), '.<BR>']))

        last_subnet_network_addr_bin = get_last_subnet_address_bin (addr_initial, mask_initial, subnet_mask, number_of_subnets)

        question.append(format_shortanswer_question(
          ''.join(["Quel est l'adresse réseau du ", str(number_of_subnets), " ème sous-réseau ? "]),
          pretty_dotted_dec(last_subnet_network_addr_bin),
          " (en décimal pointé) <BR>"))
        generalfeedback.append(''.join(["L'adresse réseau du ", str(number_of_subnets), "ème sous-réseau est ",
                                       pretty_dotted_dec(last_subnet_network_addr_bin),'.<BR>']))

        hint.append(''.join(["Le ", str(number_of_subnets), "ème sous-réseau demandé ne correspond à la valeur de ",
                              str(number_of_subnets), " en décimal mais à la valeur ",  str(number_of_subnets),"-1, soit ", str(number_of_subnets-1),
                                                                    ". En effet, en binaire le premier des sous-réseaux aura pour code ", '0'*binary_len_of_subnets, " et le ",str(number_of_subnets),"ème ", dec_to_bin(number_of_subnets-1),'.<BR>']))
        hint.append ("Pour déterminer l'adresse réseau (qui désigne le réseau), on fait un 'et logique' entre le masque et l'adresse IP en focus. Ou bien on met à zéro tous les bits de la partie machine de l'IP en focus.<BR>")

        last_broadcast_network_addr_bin = get_broadcast_address_bin (last_subnet_network_addr_bin, subnet_mask)

        question.append(format_shortanswer_question(
          ''.join(["Quel est l'adresse de diffusion du ", str(number_of_subnets), " ème sous-réseau ? "]),
          pretty_dotted_dec(last_broadcast_network_addr_bin),
          " (en décimal pointé) <BR>"))
        generalfeedback.append(''.join(["Son adresse de diffusion est ",
                                       pretty_dotted_dec(last_broadcast_network_addr_bin),'.<BR>']))
        hint.append ("Pour déterminer l'adresse de diffusion (qui permet d'adresser toutes les machines de ce réseau), on met à un tous les bits de la partie machine de l'adresse IP en focus.<BR>")


        question.append ("La plage d'adresses machines adressables s'étend de 'l'adresse réseau plus 1' en binaire à 'l'adresse de diffusion moins 1' en binaire.<BR>")
        first_host_address_bin = get_first_host_address_bin (last_subnet_network_addr_bin, subnet_mask)
        question.append(format_shortanswer_question(
          ''.join(["Quelle est la première IP utilisable pour adresser un hôte sur ce réseau ? "]),
          pretty_dotted_dec(first_host_address_bin),
          " (en décimal pointé).<BR>"))
        generalfeedback.append(''.join(["La première IP utilisable pour adresser un hôte sur ce réseau est ",
                                       pretty_dotted_dec(first_host_address_bin),'.<BR>']))

        last_host_address_bin = get_last_host_address_bin (last_subnet_network_addr_bin, subnet_mask)
        question.append(format_shortanswer_question(
          ''.join(["Quelle est la dernière IP utilisable pour adresser un hôte sur ce réseau ? "]),
          pretty_dotted_dec(last_host_address_bin),
          " (en décimal pointé).<BR>"))
        generalfeedback.append(''.join(["La dernière IP utilisable pour adresser un hôte sur ce réseau est ",
                                        pretty_dotted_dec(last_host_address_bin),'.<BR>']))
        name = ''.join([pretty_dotted_dec(network_address_bin,mask_initial), " avec ", str(number_of_subnets), " sous-réseaux de ", str(number_of_hosts), ' hôtes chacun'])

  return name, '\n'.join(question), '\n'.join(generalfeedback), '\n'.join(hint)

name, question, generalfeedback, hint =  generate_subnet_splitting()
print ('generate_subnet_splitting',name)
print (question)
print ()
print (generalfeedback)
print ()
print (hint)



generate_subnet_splitting 







* 1000 train + 1000 test en deux fois (ca fait déjà 5 Mo de data chaque fichier)
* Gestion du cours > Banque de question > importer ; créer une catégorie ; opter pour XML

In [16]:
count = 0
# SET
max_questions = 1000
wi_feedback = True # not implemented
wi_hints = False
wi_html = True # True for xml moodle format export

cloze_questions = list()
while count < max_questions:

  name, question, generalfeedback, hint =  generate_subnet_splitting()

  if len(question) !=0:
    cloze_questions.append(format_cloze_question ('subnet splitting ' + name, question, generalfeedback, hint))
    count +=1

quizz = format_moodle_quizz ('\n'.join(cloze_questions))
quizz_path = 'subnet_splitting_quizz.xml'
with open(quizz_path, "w") as text_file:
    text_file.write(quizz)

## Routing table


## Routing


In [24]:
ipv4_address_bin = generate_ipv4_address_bin()
mask_net1_cidr = generate_mask_cidr()

#ipv4_address_bin = '11001001011111001000000000000000'
#mask_net1_cidr = 17
ip_network_net1_bin = get_network_address_bin (ipv4_address_bin, mask_net1_cidr)

def get_host_id_bin(ip_address_bin, mask, ip_addr_len = 32):
  return host_id (ip_address_bin, mask, ip_addr_len = 32)

def generate_ip_host_bin_on_given_network (ipv4_address_bin, mask_cidr):
  host_id_bin_len = len(get_host_id_bin(ipv4_address_bin, mask_cidr))
  number_of_hosts = 2**host_id_bin_len -2
  rand_host_id_dec = randint(1, number_of_hosts)
  rand_host_id_bin = dec_to_bin(rand_host_id_dec)
  rand_host_id_bin = '0' * (host_id_bin_len - len(rand_host_id_bin)) + rand_host_id_bin
  return get_network_id_bin(ipv4_address_bin, mask_cidr) + rand_host_id_bin

ip_host1_on_net1_bin = generate_ip_host_bin_on_given_network (ip_network_net1_bin, mask_net1_cidr)

#print ('Debug: un réseau  :\t', pretty_dotted_dec(ip_network_net1_bin, mask_net1_cidr), '\t',pretty_dotted_bin(ip_network_net1_bin))
#print ('Debug: une machine:\t', pretty_dotted_dec(ip_host1_on_net1_bin),'\t', pretty_dotted_bin(ip_host1_on_net1_bin))
#print()

import random

def generate_following_ip_network_address(ipv4_address_bin, mask_cidr):
  #ipv4_address_bin = '01000000000000000000000000000000'
  #mask_cidr = 8
  network_id_bin = network_id (ipv4_address_bin, mask_cidr)
  #print ('Debug: len(network_id_bin)', len(network_id_bin))
  #print ('Debug: mask_cidr', mask_cidr)

  network_id_dec = int(bin_to_dec (network_id_bin))
  #print ('Debug: network_id_dec', network_id_dec)
  rand_range = 3
  following_network_id_dec = randint(network_id_dec+1,network_id_dec+1+rand_range) # arbitrairement on choisit de regarder dans une fenêtre de +rand_range
  #print ('Debug: following_network_id_dec', following_network_id_dec)
  following_network_id_bin = dec_to_bin (following_network_id_dec)
  #print ('Debug: following_network_id_bin ', following_network_id_bin)
  #print ('Debug: len(following_network_id_bin)', len(following_network_id_bin))

  following_network_id_bin = (mask_cidr-len(following_network_id_bin)) * '0' + following_network_id_bin
  #print ('Debug: following_network_id_bin post fix', following_network_id_bin)
  #print ('Debug: len(following_network_id_bin) post fix', len(following_network_id_bin))

  following_network_ip_bin = following_network_id_bin + '0' * (32-mask_cidr)
  #print ('Debug: len(following_network_ip_bin)', len(following_network_ip_bin))
  #sample_len = 3
  #print ('Debug: original_network_id_bin', network_id_bin[len(network_id_bin)-sample_len:])
  #following_network_id_bin = random.sample(network_id_bin[len(network_id_bin)-sample_len:], sample_len) # 111 gives 111 as sample...
  #following_network_id_bin = sample_len * randint(0,1)
  #print ('Debug: following_network_id_bin', following_network_id_bin)

  #following_network_ip_bin = network_id_bin[:len(network_id_bin)-sample_len] + ''.join(following_network_id_bin) + '0' * (32-mask_cidr)
  return following_network_ip_bin

#ip_network_net2_bin =  generate_following_ip_network_address(ip_network_net1_bin, mask_net1_cidr)
#ip_host1_on_net2_bin = generate_ip_host_bin_on_given_network (ip_network_net2_bin, mask_net1_cidr)
#print ('Debug: un réseau  :\t', pretty_dotted_dec(ip_network_net2_bin, mask_net1_cidr), '\t', pretty_dotted_bin(ip_network_net2_bin))
#print ('Debug: une machine:\t', pretty_dotted_dec(ip_host1_on_net2_bin), '\t', pretty_dotted_bin(ip_host1_on_net2_bin))

#print()
#ip_network_net3_bin =  generate_following_ip_network_address(ip_network_net2_bin, mask_net1_cidr)
#ip_host1_on_net3_bin = generate_ip_host_bin_on_given_network (ip_network_net3_bin, mask_net1_cidr)
#print ('Debug: un réseau  :\t', pretty_dotted_dec(ip_network_net3_bin, mask_net1_cidr), '\t',pretty_dotted_bin(ip_network_net3_bin))
#print ('Debug: une machine:\t', pretty_dotted_dec(ip_host1_on_net3_bin), '\t',pretty_dotted_bin(ip_host1_on_net3_bin))

#print()
#ip_network_net4_bin =  generate_following_ip_network_address(ip_network_net3_bin, mask_net1_cidr)
#ip_host1_on_net4_bin = generate_ip_host_bin_on_given_network (ip_network_net4_bin, mask_net1_cidr)
#print ('Debug: un réseau  :\t', pretty_dotted_dec(ip_network_net4_bin, mask_net1_cidr), '\t',pretty_dotted_bin(ip_network_net4_bin))
#print ('Debug: une machine:\t', pretty_dotted_dec(ip_host1_on_net4_bin),'\t', pretty_dotted_bin(ip_host1_on_net4_bin))

def find_destination (destination_list, mask_list, iface_list, addressee):
  # return
  # 1. the interface name for the longuest mask when multiple options, the unique interface when it is not ambiguous, or None if no matching
  # 2. some log called debug a list of [m, pretty_dotted_dec(a, m), pretty_dotted_network_id_bin(a_net_id), pretty_dotted_dec(addressee), pretty_dotted_network_id_bin(addressee_net_id), a_net_id == addressee_net_id,  iface_list[i]]
  # 3. True if the longuest mask, False in any other case
  candidate_dest = list()
  debug = list()
  i = 0
  for a, m in zip(destination_list, mask_list):
    are = are_on_the_same_network(a,addressee, m)
    addressee_net_id = get_network_id_bin(addressee, m)
    a_net_id = get_network_id_bin(a, m)
    #print (m, pretty_dotted_dec(a, m), pretty_dotted_network_id_bin(a_net_id), pretty_dotted_dec(addressee), pretty_dotted_network_id_bin(addressee_net_id), a_net_id == addressee_net_id,  iface_list[i])
    debug.append([m, pretty_dotted_dec(a, m), pretty_dotted_network_id_bin(a_net_id), pretty_dotted_dec(addressee), pretty_dotted_network_id_bin(addressee_net_id), a_net_id == addressee_net_id,  iface_list[i]])
    if are:
      candidate_dest.append(i)
    #  print ('is dest:', pretty_dotted_dec(a), '\t', pretty_dotted_bin(a, m), iface_list[i])
    #else:
    #  print ('is not :', pretty_dotted_dec(a), '\t', pretty_dotted_bin(a, m), iface_list[i])
    #debug.append([pretty_dotted_dec(a, m), m, pretty_dotted_bin(a, m), iface_list[i], are])
    i += 1
  if len(candidate_dest) >1:
    max_mask_len = -1
    iface_wi_max_mask_len = ''
    for c in candidate_dest:
      if mask_list[c] >   max_mask_len:
        max_mask_len = mask_list[c]
        iface_wi_max_mask_len = iface_list[c]
    return iface_wi_max_mask_len, debug, True
  elif len(candidate_dest) == 1:
    return iface_list[candidate_dest[0]], debug, False
  return None, debug, None

#network_addr_list = [ip_network_net1_bin, ip_network_net2_bin, ip_network_net3_bin, ip_network_net4_bin]
#mask_list = [ mask_net1_cidr, mask_net1_cidr, mask_net1_cidr, mask_net1_cidr]
#destination_list = ['eno1', 'eno2', 'eno3', 'eno4']
#addressee_list = [ip_host1_on_net1_bin, ip_host1_on_net2_bin, ip_host1_on_net3_bin, ip_host1_on_net4_bin]

#addressee = ip_host1_on_net3_bin
#print ('Debug: addressee:\t', pretty_dotted_dec(addressee),'\t', pretty_dotted_bin(addressee))
#find_destination (network_addr_list, mask_list, destination_list, addressee)

def generate_routing_table (table_size = 5):
  # create a first network
  mask_cidr = generate_mask_cidr()
  ip_addr_bin = get_network_address_bin(generate_ipv4_address_bin(), mask_cidr)
  ip_host_bin = generate_ip_host_bin_on_given_network (ip_addr_bin, mask_cidr)

  destination_list = list()
  public_destination_list = list()
  mask_list = list()
  iface_list = list()
  addressee_list = list()

  destination_list.append(ip_addr_bin)
  mask_list.append(mask_cidr)
  #iface_list.append('eno0')

  # generate 2 hosts per destination
  addressee_list.append(generate_ip_host_bin_on_given_network (ip_addr_bin, mask_cidr))
  addressee_list.append(generate_ip_host_bin_on_given_network (ip_addr_bin, mask_cidr))
  #print ('{}\t{}\t{}'.format(pretty_dotted_dec(ip_addr_bin, mask_cidr), iface_list[-1], pretty_dotted_bin(ip_addr_bin)))

  # generate subsequent networks with 2 hosts each
  for i in range (1, table_size):
    ip_addr_bin = generate_following_ip_network_address(ip_addr_bin, mask_cidr)
    destination_list.append(ip_addr_bin)
    mask_list.append(mask_cidr)
    #iface_list.append(''.join(['eno', str(i)]))
    #print ('{}\t{}\t{}'.format(pretty_dotted_dec(ip_addr_bin, mask_cidr), iface_list[-1], pretty_dotted_bin(ip_addr_bin)))
    addressee_list.append(generate_ip_host_bin_on_given_network (ip_addr_bin, mask_cidr))
    addressee_list.append(generate_ip_host_bin_on_given_network (ip_addr_bin, mask_cidr))
  #print ('Debug: len(destination_list)',len(destination_list))

  # select one to be the default and update
  # this is a fake default...
  default = randint(0, len(destination_list)-1)
  #print ('Debug: default', default)

  public_destination_list = [pretty_dotted_dec(e, m) for e,m  in zip(destination_list, mask_list) ]
  public_destination_list[default] = 'default'

  # select one to duplicate with a more specific mask and update
  more_specific_mask = randint(0, len(destination_list)-1)
  while default == more_specific_mask:
    more_specific_mask = randint(0, len(destination_list)-1)
  #print ('Debug: more_specific_mask', more_specific_mask)
  #mask_list[more_specific_mask] = mask_cidr+1 # arbitrary we inc plus one

  #public_destination_list[more_specific_mask] = pretty_dotted_dec(destination_list[more_specific_mask], mask_list[more_specific_mask])
  destination_list.append(destination_list[more_specific_mask])
  diff_mask = -1# arbitraire
  mask_list.append(mask_list[more_specific_mask]+diff_mask) # arbitrary we inc plus one
  public_destination_list.append(pretty_dotted_dec(destination_list[more_specific_mask], mask_list[more_specific_mask]+diff_mask))

  # in order to have the default at the end
  destination_list[default], destination_list[table_size] = destination_list[table_size], destination_list[default]
  public_destination_list[default], public_destination_list[table_size] = public_destination_list[table_size], public_destination_list[default]
  mask_list[default], mask_list[table_size] = mask_list[table_size], mask_list[default]
  #iface_list[default], iface_list[table_size-1] = iface_list[table_size-1], iface_list[default]

  # assigning an interface
  iface_list = [''.join(['eno', str(i)]) for i in range (0, len(destination_list))]

  return destination_list, public_destination_list, mask_list, iface_list, addressee_list

def print_routing_table (destination_list, public_destination_list, mask_list, iface_list, public = False):
  for d, pub_d, m, i in zip(destination_list, public_destination_list, mask_list, iface_list):
    if public:
      print ('{}\t{}\t{}'.format(pub_d, i, pretty_dotted_bin(d)))
    else:
      print ('{}\t{}\t{}\t{}'.format(pretty_dotted_dec(d, m), pub_d, i, pretty_dotted_bin(d)))

def print_routing_table_html (destination_list, public_destination_list, mask_list, iface_list, feedback = False):
  table = list()
  table.append('<table style="border: 1px solid black; padding: 10px; ">')
  for d, pub_d, m, i in zip(destination_list, public_destination_list, mask_list, iface_list):
    if feedback:
       if pub_d != 'default': table.append ('<tr><td>{}</td>\t<td>{}</td>\t<td>{}</td></tr>'.format(pub_d, i, pretty_dotted_bin(d))) # pretty_dotted_dec(d, m),
       else: table.append ('<tr><td>{}</td>\t<td>{}</td>\t<td>{}</td></tr>'.format(pub_d, i, '-'))
    else:
      table.append('<tr><td>{}</td>\t<td>{}</td></tr>'.format(pub_d, i))
  table.append('</table>')
  return table

table_size = 4
destination_list, public_destination_list, mask_list, iface_list, addressee_list = generate_routing_table(table_size)
print_routing_table (destination_list, public_destination_list, mask_list, iface_list)

#destination_list.append(get_network_address_bin(destination_list[0], mask_list[0]+1))
#mask_list.append(mask_list[0]+1)
#iface_list.append('enoX')
#print ('{}\t{}\t{}'.format(pretty_dotted_dec(destination_list[-1], mask_list[-1]), iface_list[-1], pretty_dotted_bin(destination_list[-1])))

print()

print ('addressee_list', [pretty_dotted_dec(a) for a in addressee_list])
#addressee_list = sorted(random.sample (addressee_list, table_size))
print()

for a in addressee_list:
  print ('Routing:', pretty_dotted_dec(a),'\t', pretty_dotted_bin(a))

  iface, debug, longest = find_destination (destination_list, mask_list, iface_list, a)
  print ('->', iface)
  #if iface:
  #  print ('->', iface)
  #else:
  #  print ('-> default')
  print ()

longuest_len = 0
ambiguous_len = 0

def generate_routing_table_question (table_size = 5):

  destination_list, public_destination_list, mask_list, iface_list, addressee_list = generate_routing_table(table_size)

  #destination_list = [ dotted_dec_to_bin('104.152.0.0'), dotted_dec_to_bin('104.168.0.0'), dotted_dec_to_bin('104.152.0.0'), dotted_dec_to_bin('104.176.0.0')]
  #public_destination_list = [ '104.152.0.0', '104.168.0.0', '104.152.0.0', 'default']
  #mask_list = [14, 14, 13, 14]
  #iface_list = ['eno0', 'eno1', 'eno2', 'eno3']
  #addressee_list= [dotted_dec_to_bin('104.153.64.13')]
  #-> en0

  question = list()
  generalfeedback = list()
  hint = list()
  name = ''

  question.append(''.join(["Soit la table de routage suivante :<BR>"]))
  question.extend(print_routing_table_html (destination_list, public_destination_list, mask_list, iface_list, feedback = False))
  question.append(''.join(["Pour chacune des adresses IP suivantes, indiquez le nom de l'interface de sortie sur laquelle le routeur aiguillera un paquet ayant une telle destination :<BR>"]))

  hint.append("Le routeur vérifie l'appartenance de l'adresse à router à chacune des destinations de sa table de routage.<BR>")
  hint.append("Pour ce faire, il fait un 'et' logique entre le masque de la destination et l'adresse à router. Il obtient une adresse réseau qu'il compare avec l'adresse réseau de la destination.<BR>")
  hint.append("Si il y a une correspondance, le paquet sera routé sur l'interface associée à la destination.<BR>")
  hint.append("En cas d'ambiguïté, il optera pour la destination avec laquelle l'adresse à router partage le plus grand identifiant réseau (i.e. avec le plus long masque).<BR>")

  # arbitrairement on ne propose qu'un nombre limité de
  addressee_list = sorted(random.sample (addressee_list, table_size+1)) # comment when debugging

  for a in addressee_list:
    generalfeedback.append(''.join(['Le paquet à destination de ', pretty_dotted_dec(a),' a pour valeur binaire : ', a, ".<BR>"]))

    iface, debug, longuest  = find_destination (destination_list, mask_list, iface_list, a)

    # some stats
    # debug.append([m, pretty_dotted_dec(a, m), pretty_dotted_network_id_bin(a_net_id), pretty_dotted_dec(addressee), pretty_dotted_network_id_bin(addressee_net_id), a_net_id == addressee_net_id,  iface_list[i]])
    #if longuest: longuest_len += 1
    #ambiguous = 0
    #longuest_mask = -1
    #for m, p_dest_dec, p_dest_bin, p_addressee, p_net_id, is_match, output_iface in debug:
    #  if is_match: ambiguous += 1
    #if ambiguous >1:
    #  ambiguous_len +=1

    #
    alternative_answers = list()
    i = 0
    for m, p_dest_dec, p_dest_bin, p_addressee, p_net_id, is_match, output_iface in debug:
      if not (output_iface in alternative_answers):
        alternative_answers.append(output_iface)
        if output_iface == iface:
            true_one = i
        i+=1
    question.append(format_multichoice_question(''.join(["Le paquet ayant l'adresse IP de destination suivante ", pretty_dotted_dec(a) ," sera aiguillé sur l'interface : "]),
                                                alternative_answers,
                                                true_one,
                                                "<BR>"))

    #question.append(format_shortanswer_question(
    #      ''.join(["Le paquet ayant l'adresse IP de destination suivante ", pretty_dotted_dec(a) ," sera aiguillé sur l'interface : "]),
    #      iface,
    #      "<BR>"))
    #generalfeedback.append(''.join([str(available_bits), ' bits sont donc suffisants pour couvrir ces besoins.<BR>']))
    #hint.append(''.join([pretty_dotted_dec(a), " se convertit en binaire en ",,".<BR>"]))
  #print(pretty_dotted_dec(a), '->', iface)
    for i, d in enumerate(debug):
      #       debug.append([m, pretty_dotted_dec(a, m), pretty_dotted_network_id_bin(a_net_id), pretty_dotted_dec(addressee), pretty_dotted_network_id_bin(addressee_net_id), a_net_id == addressee_net_id,  iface_list[i]])
      if i+1 == len(debug):
        generalfeedback.append(''.join(["Le default accepte toutes les destinations.<BR>"]))
      else:
        generalfeedback.append(''.join(["L'identifiant réseau de ", d[1], " est ", d[2], " en binaire. Quand on applique son masque ", str(d[0]), " sur l'IP à router, on obtient l'identifiant binaire ",  d[4], ".<BR>"]))
        if d[5]: generalfeedback.append('Ces identifiants sont identifiques. <BR>')
        else: generalfeedback.append('Ces identifiants ne sont pas identifiques. <BR>')
    if longest:
      generalfeedback.append(''.join(['Il y plusieurs destinations candidates. Le routeur opte pour celle avec le masque le plus long, soit ',iface,'. <BR>']))
    else:
            generalfeedback.append(''.join(["Il n'y a pas d'ambiguïté. Le routeur opte pour l'interface ",iface,'. <BR>']))
    generalfeedback.append('<BR>')

  name = pretty_dotted_dec(addressee_list[0]) +' /'+ str(mask_list[0])
  return name, '\n'.join(question), '\n'.join(generalfeedback), '\n'.join(hint)



162.192.0.0/11	162.192.0.0/11	eno0	1010 0010 . 1100 0000 . 0000 0000 . 0000 0000
163.32.0.0/11	163.32.0.0/11	eno1	1010 0011 . 0010 0000 . 0000 0000 . 0000 0000
163.224.0.0/10	163.224.0.0/10	eno2	1010 0011 . 1110 0000 . 0000 0000 . 0000 0000
163.224.0.0/11	163.224.0.0/11	eno3	1010 0011 . 1110 0000 . 0000 0000 . 0000 0000
163.160.0.0/11	default	eno4	1010 0011 . 1010 0000 . 0000 0000 . 0000 0000

addressee_list ['162.200.235.143', '162.199.225.37', '163.35.244.76', '163.54.11.173', '163.164.56.57', '163.177.152.247', '163.232.136.146', '163.252.24.68']

Routing: 162.200.235.143 	 1010 0010 . 1100 1000 . 1110 1011 . 1000 1111
-> eno0

Routing: 162.199.225.37 	 1010 0010 . 1100 0111 . 1110 0001 . 0010 0101
-> eno0

Routing: 163.35.244.76 	 1010 0011 . 0010 0011 . 1111 0100 . 0100 1100
-> eno1

Routing: 163.54.11.173 	 1010 0011 . 0011 0110 . 0000 1011 . 1010 1101
-> eno1

Routing: 163.164.56.57 	 1010 0011 . 1010 0100 . 0011 1000 . 0011 1001
-> eno4

Routing: 163.177.152.247 	 1010 0011 . 1

104.152.0.0/14 0110 1000 . 1001 10 104.153.64.13 0110 1000 . 1001 10 True eno0



14 104.168.0.0/14 0110 1000 . 1010 10 104.153.64.13 0110 1000 . 1001 10 False eno1
13 104.152.0.0/13 0110 1000 . 1001 1 104.153.64.13 0110 1000 . 1001 1

In [26]:
count = 0
# SET
max_questions = 500
wi_feedback = True # not implemented
wi_hints = False
wi_html = True # True for xml moodle format export

cloze_questions = list()
while count < max_questions:

  table_size = 4
  name, question, generalfeedback, hint = generate_routing_table_question(table_size)

  #name, question, generalfeedback, hint =  generate_subnet_splitting()

  if len(question) !=0:
    cloze_questions.append(format_cloze_question ('Routing packet ' + name, question, generalfeedback, hint))
    count +=1

quizz = format_moodle_quizz ('\n'.join(cloze_questions))
quizz_path = 'routing_quizz.xml'
with open(quizz_path, "w") as text_file:
    text_file.write(quizz)